In [ ]:
path = "/home"

# multi reference rougel for SQuALITY

In [ ]:
import json
from rouge_score import rouge_scorer
import json

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL', 'rougeLsum'], use_stemmer=True, split_summaries=True)
with open(f'{path}/outputs/LLaMA3-Lora-hyper/SQuALITY/b32_epoch6_warme1_lorar8_loraQ,K,V,O,FFN_UP_nhyper16-32_diffe_parallel_blr6e3_lossOlabelsTrue_maxseq8000_flashattnTrue_/predict_mingen200.jsonl', 'r') as f:
    lines = f.readlines()

data = [json.loads(line) for line in lines]

idset = []
new_data = {}
for x in data:
    if x.get('instruction'):
        ins_art = x.get('instruction')+x.get('article')
    else:
        ins_art = x.get('article')

    if ins_art in idset:
        x['idx'] = idset.index(ins_art)
        new_data[x['idx']].append(x)
    else:
        idset.append(ins_art)
        x['idx'] = len(idset)-1
        new_data[x['idx']] = [x]

In [ ]:
new_data[1]

In [ ]:
def rougel(abstract, generate):
    generate = generate.replace('<pad>','').replace('<s>','').replace('</s>','')
    scores = scorer.score(abstract, generate)
    rouge1 = scores['rouge1'].fmeasure
    rouge2 = scores['rouge2'].fmeasure
    rougeL = scores['rougeL'].fmeasure
    rougeLsum = scores['rougeLsum'].fmeasure
    return rouge1, rouge2, rougeL, rougeLsum

In [ ]:
cnt = 0
rouge1, rouge2, rougeL, rougeLsum= 0, 0, 0, 0
for k,v in new_data.items():
    # print(len(v))
    r1_l = []
    r2_l = []
    rL_l = []
    rLsum_l = []
    for one in v:
        r1, r2, rL, rLsum = rougel(one['abstract'], one['generate'])
        r1_l.append(r1)
        r2_l.append(r2)
        rL_l.append(rL)
        rLsum_l.append(rLsum)
    max_idx = rL_l.index(max(rL_l))
    rouge1 += r1_l[max_idx]
    rouge2 += r2_l[max_idx]
    rougeL += rL_l[max_idx]
    rougeLsum += rLsum_l[max_idx]
    # print(rL_l)
    # print(rLsum_l)
    # print(rougeL)
    cnt += 1
    # break
print(f"rouge1: {rouge1 / cnt}, rouge2: {rouge2 / cnt}, rougeL: {rougeL / cnt}, rougeLsum: {rougeLsum / cnt}")


# multi reference bart_score for SQuALITY

In [ ]:
from bert_score import BERTScorer

Bert_scorer = BERTScorer(model_type=f'{path}/pretrain_models/bart_base', num_layers=6,device='cuda:0',
# Bert_scorer = BERTScorer(model_type=f'{path}/pretrain_models/deberta-xlarge-mnli', num_layers=40,device='cuda:0', # is not the bertscore in Qontsum
                    batch_size=256,
                    nthreads=8,
                    # idf=True,
                    # rescale_with_baseline=True,
                    # lang="en"
                    # idf_sents=refs
                    )

In [ ]:

all_ref = []
all_gen = []
number_l = []
for k,v in new_data.items():
    number_l.append(len(v))
    for one in v:
        all_ref.append(one['abstract'])
        all_gen.append(one['generate'].replace('<pad>','').replace('<s>','').replace('</s>',''))

len(number_l)

_, _, f1 = Bert_scorer.score(all_gen, all_ref)

cnt = 0
bart_score= 0
ind = 0
for numb in number_l:
    barts_l = list(f1[ind:ind+numb])
    max_barts = max(barts_l)
    bart_score += max_barts
    cnt += 1
    ind = ind+numb
print(f"bart_score: {bart_score / cnt}")